# US CRIME ANALYSIS

# PHASE 1: ENVIRONMENT SETUP

In [2]:
import requests
import time as t
import pandas as pd
import datetime
import pprint
from datetime import date 
from bs4 import BeautifulSoup

import glob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as po

import statistics as stats
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# from collections import Counter
# from string import punctuation

In [3]:
program_start = t.time()

#  PHASE 2: ETL

In [8]:
# import listings-2.csv into pandas dataframe
data_raw = pd.read_csv("data/crime_first_half.csv", index_col=0) 
data_raw.head(5)

,X,REPORT_DAT,SHIFT,OFFENSE,METHOD,BLOCK,DISTRICT,PSA,WARD,ANC,...,year,month,day,hour,minute,second,EW,NS,quad,crimetype
1,1,8/31/08 20:47,EVENING,THEFT/OTHER,OTHERS,3500 - 3599 BLOCK OF R STREET NW,2.0,206.0,2,2E,...,2008,8,31,20,47,0,West,North,Northwest,Non-Violent
2,2,9/1/08 0:45,MIDNIGHT,MOTOR VEHICLE THEFT,OTHERS,2000 - 2015 BLOCK OF O STREET NW,2.0,208.0,2,2B,...,2008,9,1,0,45,0,West,North,Northwest,Non-Violent
3,3,9/1/08 3:00,MIDNIGHT,THEFT/OTHER,OTHERS,700 - 799 BLOCK OF 11TH STREET NW,1.0,101.0,2,2C,...,2008,9,1,3,0,0,East,North,Northeast,Non-Violent
4,4,9/9/08 7:46,DAY,THEFT/OTHER,OTHERS,1700 - 1799 BLOCK OF P STREET NW,2.0,208.0,2,2B,...,2008,9,9,7,46,0,West,North,Northwest,Non-Violent
5,5,8/24/08 20:00,EVENING,MOTOR VEHICLE THEFT,OTHERS,500 - 599 BLOCK OF INDIANA AVENUE NW,1.0,102.0,2,2C,...,2008,8,24,20,0,0,East,North,Northeast,Non-Violent


In [12]:
data_raw.describe()

,X,DISTRICT,PSA,WARD,CENSUS_TRACT,CCN,XBLOCK,YBLOCK,year,month,day,hour,minute,second
count,169998.000000,169886.000000,169855.000000,169998.000000,169794.000000,1.699980e+05,169998.000000,169998.000000,169998.000000,169998.000000,169998.000000,169998.000000,169998.000000,169998.000000
mean,84999.500000,3.059811,311.041959,3.579219,5583.778667,1.248476e+07,-77.024401,38.922111,2012.374851,6.638760,15.919476,13.311515,26.240462,6.326398
std,49074.339868,1.439088,143.534566,1.404180,3576.479169,2.835537e+06,0.030272,0.023613,2.798939,3.330802,8.768878,6.033268,18.097033,14.503669
min,1.000000,1.000000,101.000000,2.000000,100.000000,1.000600e+05,-77.113642,38.859656,2008.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,42500.250000,2.000000,205.000000,2.000000,2101.000000,1.009567e+07,-77.042577,38.904524,2010.000000,4.000000,8.000000,9.000000,10.000000,0.000000
50%,84999.500000,3.000000,305.000000,4.000000,5500.000000,1.214518e+07,-77.026000,38.914086,2012.000000,7.000000,16.000000,14.000000,28.000000,0.000000
75%,127498.750000,4.000000,407.000000,5.000000,9102.000000,1.506708e+07,-77.000935,38.939801,2015.000000,9.000000,23.000000,18.000000,42.000000,0.000000
max,169998.000000,7.000000,706.000000,6.000000,11100.000000,9.902096e+07,-76.949527,38.994901,2017.000000,12.000000,31.000000,23.000000,59.000000,59.000000


In [13]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169998 entries, 1 to 169998
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   X                     169998 non-null  int64  
 1   REPORT_DAT            169998 non-null  object 
 2   SHIFT                 169998 non-null  object 
 3   OFFENSE               169998 non-null  object 
 4   METHOD                169998 non-null  object 
 5   BLOCK                 169998 non-null  object 
 6   DISTRICT              169886 non-null  float64
 7   PSA                   169855 non-null  float64
 8   WARD                  169998 non-null  int64  
 9   ANC                   169998 non-null  object 
 10  NEIGHBORHOOD_CLUSTER  168164 non-null  object 
 11  BLOCK_GROUP           169794 non-null  object 
 12  CENSUS_TRACT          169794 non-null  float64
 13  VOTING_PRECINCT       169963 non-null  object 
 14  CCN                   169998 non-null  int64  
 15  

In [9]:
col_list = list(data_raw.columns)
col_list

['X',
 'REPORT_DAT',
 'SHIFT',
 'OFFENSE',
 'METHOD',
 'BLOCK',
 'DISTRICT',
 'PSA',
 'WARD',
 'ANC',
 'NEIGHBORHOOD_CLUSTER',
 'BLOCK_GROUP',
 'CENSUS_TRACT',
 'VOTING_PRECINCT',
 'CCN',
 'START_DATE',
 'END_DATE',
 'XBLOCK',
 'YBLOCK',
 'optional',
 'date',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'EW',
 'NS',
 'quad',
 'crimetype']

In [10]:
col_list_length = len(col_list)
col_list_length

31